In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
!pip freeze | grep boto
!pip freeze | grep agentcore

In [1]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

In [2]:
![ ! -d "agents" ] && mkdir agents

In [3]:
%%writefile agents/settlement-agent.py
import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

from typing import Dict, Any, List
from dataclasses import dataclass



print("✓ All dependencies imported successfully")




logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


# AWS Configuration
AWS_REGION = "us-east-1"
MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)

print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")



# Initialize Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
)


@dataclass
class CoveragePolicy:
    """Policy coverage configuration"""
    collision_coverage: float = 50000.0
    deductible: float = 500.0
    policy_number: str = "POL-2023-456789"
    policy_status: str = "ACTIVE"
    coverage_type: str = "COMPREHENSIVE"


class AutoInsuranceClaimsAgent:
    """
    AWS Strands Agent for Auto Insurance Claims Settlement Automation
    Handles complete claim processing from verification to settlement
    """
    
    def __init__(self):
        self.policy_database = {
            "CL-2023-1156789": CoveragePolicy(
                collision_coverage=50000.0,
                deductible=500.0,
                policy_number="POL-2023-456789",
                policy_status="ACTIVE",
                coverage_type="COMPREHENSIVE"
            )
        }
        self.claim_data = None
        self.coverage_info = None
        self.settlement_details = {}
    
    @tool
    def verify_coverage_limits(self, claim_number: str, estimated_cost: float) -> Dict[str, Any]:
        """
        Verifies applicable coverage limits against policy terms.
        
        Args:
            claim_number: Unique claim identifier
            estimated_cost: Total estimated repair cost
            
        Returns:
            Coverage verification details with policy limits and applicability
        """
        policy = self.policy_database.get(claim_number)
        
        if not policy:
            return {
                "status": "ERROR",
                "message": f"No policy found for claim {claim_number}",
                "coverage_applicable": False
            }
        
        if policy.policy_status != "ACTIVE":
            return {
                "status": "DENIED",
                "message": f"Policy status is {policy.policy_status}",
                "coverage_applicable": False,
                "policy_citation": f"Policy {policy.policy_number} - Section 2.1: Coverage requires active policy status"
            }
        
        coverage_applicable = estimated_cost <= policy.collision_coverage
        
        coverage_result = {
            "status": "VERIFIED",
            "coverage_applicable": coverage_applicable,
            "policy_number": policy.policy_number,
            "coverage_type": policy.coverage_type,
            "coverage_limit": policy.collision_coverage,
            "estimated_cost": estimated_cost,
            "deductible": policy.deductible,
            "within_limits": coverage_applicable,
            "excess_amount": max(0, estimated_cost - policy.collision_coverage)
        }
        
        self.coverage_info = coverage_result
        return coverage_result
    
    @tool
    def apply_deductible_adjustments(self, estimated_cost: float, deductible: float) -> Dict[str, Any]:
        """
        Applies policy deductible to calculate net insurance payout.
        
        Args:
            estimated_cost: Total repair estimate
            deductible: Policy deductible amount
            
        Returns:
            Detailed breakdown of adjusted amounts
        """
        adjusted_amount = max(0, estimated_cost - deductible)
        
        adjustment_details = {
            "original_estimate": estimated_cost,
            "deductible_amount": deductible,
            "adjusted_payout": adjusted_amount,
            "policyholder_responsibility": min(estimated_cost, deductible),
            "insurance_responsibility": adjusted_amount,
            "adjustment_applied": True
        }
        
        self.settlement_details.update(adjustment_details)
        return adjustment_details
    
    @tool
    def evaluate_claim_decision(self, claim_data: Dict[str, Any], coverage_info: Dict[str, Any], 
                                adjusted_amount: float) -> Dict[str, Any]:
        """
        Makes claim decision: APPROVE, DENY, or INVESTIGATE based on policy rules and risk factors.
        
        Args:
            claim_data: Complete claim information
            coverage_info: Coverage verification results
            adjusted_amount: Net payout after deductible
            
        Returns:
            Decision outcome with reasoning and conditions
        """
        decision = {
            "claim_number": claim_data.get("estimate_metadata", {}).get("claim_number"),
            "decision_date": datetime.now().isoformat(),
            "decision_by": "AUTO_CLAIMS_SETTLEMENT_AGENT"
        }
        
        investigation_triggers = []
        
        # Check for high-risk damage indicators
        damage_components = claim_data.get("damage_assessment", {}).get("damaged_components", [])
        for component in damage_components:
            damage_desc = component.get("damage_description", "").lower()
            if "frame damage" in damage_desc or "structural" in damage_desc:
                investigation_triggers.append(
                    f"Possible frame/structural damage detected in {component.get('component')} - requires certified inspection"
                )
        
        # Check auto-approval threshold
        if adjusted_amount > 5000:
            investigation_triggers.append(
                f"Claim amount \${adjusted_amount:,.2f} exceeds auto-approval threshold of \$5,000"
            )
        
        # DENY: Coverage not applicable
        if not coverage_info.get("coverage_applicable", False):
            decision["outcome"] = "DENY"
            decision["reasoning"] = "Claim amount exceeds maximum policy coverage limits"
            decision["policy_citations"] = [
                f"Policy {coverage_info.get('policy_number')} - Section 4.2: Collision coverage limit is \${coverage_info.get('coverage_limit'):,.2f}",
                f"Claim estimate \${coverage_info.get('estimated_cost'):,.2f} exceeds coverage by \${coverage_info.get('excess_amount'):,.2f}"
            ]
            decision["details"] = "The estimated repair costs exceed the maximum coverage limit specified in your policy. You may appeal this decision with additional documentation."
            return decision
        
        # INVESTIGATE: Risk factors present
        if investigation_triggers:
            decision["outcome"] = "INVESTIGATE"
            decision["reasoning"] = "Claim requires additional investigation before settlement authorization"
            decision["investigation_requirements"] = investigation_triggers
            decision["required_actions"] = [
                "Schedule comprehensive vehicle inspection by certified adjuster",
                "Obtain structural integrity assessment from authorized facility",
                "Document all hidden damage through detailed photographic evidence",
                "Verify repair estimates from manufacturer-certified repair center",
                "Review vehicle history for pre-existing damage"
            ]
            decision["estimated_investigation_timeline"] = "3-5 business days"
            decision["next_steps"] = "Our claims adjuster will contact you within 24 hours to schedule inspection"
            return decision
        
        # APPROVE: All criteria met
        decision["outcome"] = "APPROVE"
        decision["reasoning"] = "Claim meets all policy criteria and approval thresholds"
        decision["approval_conditions"] = [
            "Payment subject to submission of final itemized repair invoice",
            "Repairs must be completed at insurance-approved or certified repair facility",
            "Policyholder responsible for deductible payment directly to repair facility",
            "Final quality inspection may be required before claim closure",
            "Supplemental claims for hidden damage must be submitted within 30 days of initial repair"
        ]
        decision["approved_amount"] = adjusted_amount
        decision["payment_method"] = "Direct deposit to policyholder or two-party check to policyholder and repair facility"
        
        return decision
    
    @tool
    def create_claim_documentation(self, claim_data: Dict[str, Any], coverage_info: Dict[str, Any],
                                   adjustment_details: Dict[str, Any], decision: Dict[str, Any]) -> Dict[str, Any]:
        """
        Creates comprehensive audit documentation for regulatory compliance and future reference.
        
        Args:
            claim_data: Original claim submission
            coverage_info: Coverage verification results
            adjustment_details: Financial adjustments
            decision: Claim decision details
            
        Returns:
            Complete documentation package with audit trail
        """
        metadata = claim_data.get("estimate_metadata", {})
        vehicle_info = claim_data.get("vehicle_information", {})
        damage_assessment = claim_data.get("damage_assessment", {})
        
        documentation = {
            "document_type": "AUTO_INSURANCE_CLAIM_SETTLEMENT_RECORD",
            "document_id": f"DOC-{metadata.get('claim_number')}-{datetime.now().strftime('%Y%m%d%H%M%S')}",
            "generated_date": datetime.now().isoformat(),
            "regulatory_compliance": "DOI-2023-Standards-Compliant",
            
            "claim_summary": {
                "claim_number": metadata.get("claim_number"),
                "claim_date": metadata.get("estimate_date"),
                "estimate_disclaimer": metadata.get("disclaimer"),
                "vehicle": {
                    "year": vehicle_info.get("year"),
                    "make": vehicle_info.get("make"),
                    "model": vehicle_info.get("model"),
                    "vin": vehicle_info.get("vin"),
                    "mileage": vehicle_info.get("mileage"),
                    "type": vehicle_info.get("vehicle_type")
                },
                "damage_summary": {
                    "total_components_damaged": len(damage_assessment.get("damaged_components", [])),
                    "components": [
                        {
                            "component": comp.get("component"),
                            "description": comp.get("damage_description"),
                            "estimated_cost": comp.get("estimated_cost")
                        }
                        for comp in damage_assessment.get("damaged_components", [])
                    ],
                    "damage_subtotal": damage_assessment.get("subtotal"),
                    "total_estimate": claim_data.get("total_estimate", {}).get("amount")
                }
            },
            
            "coverage_verification": coverage_info,
            "financial_breakdown": adjustment_details,
            "decision_record": decision,
            
            "audit_trail": {
                "processed_by": "AWS_STRANDS_AUTO_CLAIMS_AGENT_v1.0",
                "processing_timestamp": datetime.now().isoformat(),
                "automated_decision": True,
                "verification_steps_completed": [
                    "✓ Policy status and coverage limits verified",
                    "✓ Deductible adjustments calculated and applied",
                    "✓ Risk assessment completed",
                    "✓ Decision matrix evaluation performed",
                    "✓ Compliance documentation generated"
                ],
                "data_sources": ["Policy Database", "Claim Intake System", "Risk Assessment Engine"]
            }
        }
        
        return documentation
    
    @tool
    def generate_settlement_output(self, documentation: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generates formatted settlement summary with all required output components.
        
        Args:
            documentation: Complete claim documentation
            
        Returns:
            Structured settlement output ready for disbursement processing
        """
        decision = documentation.get("decision_record", {})
        financial = documentation.get("financial_breakdown", {})
        claim_summary = documentation.get("claim_summary", {})
        coverage = documentation.get("coverage_verification", {})
        
        output = {
            "SETTLEMENT_SUMMARY": {
                "claim_number": claim_summary.get("claim_number"),
                "decision": decision.get("outcome"),
                "decision_date": decision.get("decision_date"),
                "vehicle": f"{claim_summary['vehicle']['year']} {claim_summary['vehicle']['make']} {claim_summary['vehicle']['model']}",
                "vin": claim_summary['vehicle']['vin'],
                
                "line_item_breakdown": [
                    {
                        "item": comp.get("component"),
                        "damage_description": comp.get("description"),
                        "estimated_cost": f"\${comp.get('estimated_cost'):,.2f}"
                    }
                    for comp in claim_summary.get("damage_summary", {}).get("components", [])
                ],
                
                "financial_summary": {
                    "total_damage_estimate": f"\${financial.get('original_estimate', 0):,.2f}",
                    "policy_deductible": f"\${financial.get('deductible_amount', 0):,.2f}",
                    "net_insurance_payout": f"\${financial.get('adjusted_payout', 0):,.2f}",
                    "policyholder_responsibility": f"\${financial.get('policyholder_responsibility', 0):,.2f}"
                },
                
                "coverage_details": {
                    "policy_number": coverage.get("policy_number"),
                    "coverage_type": coverage.get("coverage_type"),
                    "coverage_limit": f"\${coverage.get('coverage_limit', 0):,.2f}"
                }
            },
            
            "PAYMENT_AUTHORIZATION": self._generate_payment_authorization(decision, financial, claim_summary),
            
            "REQUIRED_DOCUMENTATION": {
                "mandatory_documents": self._generate_required_docs(decision),
                "submission_method": "Upload via policyholder portal or email to claims@insurance.com",
                "submission_deadline": "Within 30 days of claim approval"
            },
            
            "CONDITIONS_AND_REQUIREMENTS": {
                "decision_specific": decision.get("approval_conditions", 
                                                 decision.get("investigation_requirements", 
                                                            decision.get("policy_citations", []))),
                "general_terms": [
                    "All repairs must be completed within 90 days of approval",
                    "Salvage or total loss determination may supersede this estimate",
                    "Depreciation may apply to parts over 3 years old per policy terms"
                ]
            },
            
            "NEXT_STEPS": self._generate_next_steps(decision),
            
            "CONTACT_INFORMATION": {
                "claims_hotline": "1-800-CLAIMS-1",
                "email": "claims@insurance.com",
                "claims_adjuster": "Will be assigned within 24 hours",
                "online_portal": "https://claims.insurance.com"
            }
        }
        
        return output
    
    def _generate_payment_authorization(self, decision: Dict, financial: Dict, claim_summary: Dict) -> Dict[str, Any]:
        """Generate payment authorization details based on decision outcome"""
        if decision.get("outcome") == "APPROVE":
            return {
                "authorization_status": "✓ AUTHORIZED",
                "authorized_amount": f"\${financial.get('adjusted_payout', 0):,.2f}",
                "authorization_code": f"AUTH-{claim_summary.get('claim_number')}-{datetime.now().strftime('%Y%m%d')}",
                "payment_method": decision.get("payment_method"),
                "disbursement_timeline": "5-7 business days after receipt of required documentation",
                "payee_options": [
                    "Direct deposit to policyholder bank account",
                    "Two-party check (policyholder and repair facility)",
                    "Direct payment to approved repair facility"
                ],
                "authorization_valid_until": "90 days from authorization date"
            }
        elif decision.get("outcome") == "INVESTIGATE":
            return {
                "authorization_status": "⚠ PENDING INVESTIGATION",
                "authorized_amount": "\$0.00 (Pending)",
                "status": "Payment authorization withheld pending investigation completion",
                "reason": decision.get("reasoning"),
                "estimated_resolution": decision.get("estimated_investigation_timeline"),
                "provisional_amount": f"\${financial.get('adjusted_payout', 0):,.2f} (subject to adjustment)"
            }
        else:  # DENY
            return {
                "authorization_status": "✗ DENIED",
                "authorized_amount": "\$0.00",
                "denial_reason": decision.get("reasoning"),
                "policy_citations": decision.get("policy_citations", []),
                "appeal_rights": "You have the right to appeal this decision within 60 days",
                "appeal_contact": "appeals@insurance.com or 1-800-APPEAL-1"
            }
    
    def _generate_required_docs(self, decision: Dict) -> List[str]:
        """Generate list of required documentation based on decision"""
        base_docs = [
            "✓ Completed claim form with original signature",
            "✓ Copy of driver's license or government-issued ID",
            "✓ Vehicle registration and insurance card",
            "✓ Photographs of all damaged areas (minimum 6 angles)",
            "✓ Police report or incident report (if applicable)"
        ]
        
        if decision.get("outcome") == "APPROVE":
            base_docs.extend([
                "✓ Final itemized repair invoice on shop letterhead",
                "✓ Proof of deductible payment receipt",
                "✓ Before and after repair photographs",
                "✓ Parts receipts and warranty information",
                "✓ Odometer statement at time of repair"
            ])
        elif decision.get("outcome") == "INVESTIGATE":
            base_docs.extend([
                "✓ Comprehensive professional inspection report",
                "✓ Frame/structural integrity certification",
                "✓ Detailed photographs of suspected structural damage",
                "✓ Original repair estimate from certified facility",
                "✓ Vehicle history report (CARFAX or AutoCheck)"
            ])
        
        return base_docs
    
    def _generate_next_steps(self, decision: Dict) -> List[str]:
        """Generate next steps based on decision outcome"""
        if decision.get("outcome") == "APPROVE":
            return [
                "1. Select an approved repair facility or use your preferred shop",
                "2. Submit required documentation via online portal",
                "3. Pay your deductible directly to the repair facility",
                "4. Authorize repairs to begin",
                "5. Payment will be issued upon verification of completed repairs"
            ]
        elif decision.get("outcome") == "INVESTIGATE":
            return [
                "1. Wait for claims adjuster to contact you within 24 hours",
                "2. Schedule vehicle inspection at mutually convenient time",
                "3. Provide access to vehicle for comprehensive assessment",
                "4. Submit any additional documentation requested",
                "5. Decision will be updated within 3-5 business days after inspection"
            ]
        else:  # DENY
            return [
                "1. Review the denial reason and policy citations provided",
                "2. Gather any additional evidence to support your claim",
                "3. Contact our appeals department if you wish to appeal",
                "4. Submit appeal with supporting documentation within 60 days",
                "5. Consider alternative coverage options if applicable"
            ]



agent_instance = AutoInsuranceClaimsAgent()


agent_instructions="""
        You are an expert Auto Insurance Claims Settlement Agent powered by AWS Strands.
        
        Your responsibilities:
        1. Verify coverage limits and policy compliance
        2. Calculate accurate deductible adjustments
        3. Make fair and policy-compliant decisions (APPROVE/DENY/INVESTIGATE)
        4. Generate comprehensive audit documentation
        5. Produce clear, actionable settlement summaries
        
        Decision Guidelines:
        - APPROVE claims under \$5K with no structural damage
        - INVESTIGATE claims with frame damage, structural issues, or amounts >\$5K
        - DENY claims exceeding coverage limits with proper policy citations
        
        Always prioritize accuracy, fairness, regulatory compliance, and customer clarity.
        """
# settlement-agent.py
settlement_agent = Agent(
        name="AutoInsuranceClaimsSettlementAgent",
        description="Intelligent agent for automated auto insurance claims processing and settlement decisions",
        model=model,
        system_prompt=agent_instructions,
        tools=[
            agent_instance.verify_coverage_limits,
            agent_instance.apply_deductible_adjustments,
            agent_instance.evaluate_claim_decision,
            agent_instance.create_claim_documentation,
            agent_instance.generate_settlement_output
        ]
    )

################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=settlement_agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################




Overwriting agents/settlement-agent.py


In [4]:
# # Run the agent
# response = settlement_agent(user_prompt)

# print("\n" + "=" * 80)
# print("✓ Agent execution completed successfully")
# print("=" * 80)

In [5]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_K101vl1qr
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_K101vl1qr/.well-known/openid-configuration
Client ID: 2meg59a3s27qan0obh7bp5go7u
Bearer Token: eyJraWQiOiJRa25YSDV5cEV0Uyt3V2FqMkJKK0xwZm55QVdRRlRkTDIxKzJQbEkxZFwvWT0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiJhNGY4MzQxOC1lMGExLTcwMzYtMzU4Ni1lN2JhYmRlMmIyOTUiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9LMTAxdmwxcXIiLCJjbGllbnRfaWQiOiIybWVnNTlhM3MyN3FhbjBvYmg3YnA1Z283dSIsIm9yaWdpbl9qdGkiOiIwNmQwZDM3My1mY2Q2LTQ3YzAtYjY0OS04YmMzNmY0YTNhZmEiLCJldmVudF9pZCI6ImE0Y2YwNzYzLTA1MTMtNDI1OS1iYzMyLTU0ZTE3N2Y0MjU3OCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NjIyMTQ3NjIsImV4cCI6MTc2MjIxODM2MiwiaWF0IjoxNzYyMjE0NzYyLCJqdGkiOiJkN2E0OTY5Yi0wZThmLTRlMjAtYWExNy1lZDhjZGI0NDU3ZTIiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.J4wC9dLaBcurgIz715_NJEad_nyH_ord_Tn0F1TOjQjpkAtEeMFGeCzw_zSQEVXs8XilvMgXagu

In [6]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_SETTLEMENT_ROLE_NAME

execution_role_arn_settlement = create_agentcore_runtime_execution_role(AWS_SETTLEMENT_ROLE_NAME)

ℹ️ Role AWSDocsAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSDocsAssistantBedrockAgentCoreRole-us-east-1


In [7]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_settlement_agent = Runtime()
settlement_agent_name="settlement_assistant"

region = boto_session.region_name

# Configure the deployment
response_settlement_agent = agentcore_runtime_settlement_agent.configure(
    entrypoint="agents/settlement-agent.py",
    execution_role=execution_role_arn_settlement,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=settlement_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_settlement_agent)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/settlement-agent.py, bedrock_agentcore_name=settlement-agent
Memory configured with STM only
Configuring BedrockAgentCore agent: settlement_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only
Found existing memory ID from previous launch: settlement_assistant_mem-O5FbW18Zxl


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: Dockerfile
Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Changing default agent from 'aws_appraisal_assistant' to 'settlement_assistant'
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSDocsAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


In [8]:
launch_result_settlement= agentcore_runtime_settlement_agent.launch()
print("Launch completed:", launch_result_settlement.agent_arn)

settlement_agent_arn = launch_result_settlement.agent_arn

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: settlement_assistant
✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: settlement_assistant_mem-O5FbW18Zxl...
  Found memory: settlement_assistant_mem-O5FbW18Zxl
Found existing memory in cloud: settlement_assistant_mem-O5FbW18Zxl
Existing memory has 0 strategies
✅ Using existing STM-only memory
Starting CodeBuild ARM64 deployment for agent 'settlement_assistant' to account 161615149547 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: settlement_assistant

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-settlement_assistant


Getting or creating CodeBuild execution role for agent: settlement_assistant
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-388ccdee1d
Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-388ccdee1d
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: settlement_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-settlement_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 9.3s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 12s)
✅ BUILD completed in 18.6s
🔄 POST_BUILD started (total: 31s)
✅ POST_BUILD completed in 13.4s
🔄 COMPLETED started (total: 44s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 45s
CodeBuild co

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/settlement_assistant-3t3GvOE7iu


In [9]:
status_response = agentcore_runtime_settlement_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: settlement_assistant_mem-O5FbW18Zxl...
  Found memory: settlement_assistant_mem-O5FbW18Zxl
Retrieved Bedrock AgentCore status for: settlement_assistant


Final status: READY


#### Export and save outputs

Export variables to be used in next notebooks:

In [11]:
SETTLEMENT_AGENT_ID = launch_result_settlement.agent_id
SETTLEMENT_AGENT_ARN = launch_result_settlement.agent_arn
SETTLEMENT_AGENT_NAME = settlement_agent_name


SETTLEMENT_COGNITO_CLIENT_ID = cognito_config.get("client_id")
SETTLEMENT_COGNITO_SECRET = cognito_config.get("client_secret")
SETTLEMENT_DISCOVERY_URL = cognito_config.get("discovery_url")

%store SETTLEMENT_AGENT_ID
%store SETTLEMENT_AGENT_ARN
%store SETTLEMENT_AGENT_NAME
%store SETTLEMENT_COGNITO_CLIENT_ID
%store SETTLEMENT_COGNITO_SECRET
%store SETTLEMENT_DISCOVERY_URL

Stored 'SETTLEMENT_AGENT_ID' (str)
Stored 'SETTLEMENT_AGENT_ARN' (str)
Stored 'SETTLEMENT_AGENT_NAME' (str)
Stored 'SETTLEMENT_COGNITO_CLIENT_ID' (str)
Stored 'SETTLEMENT_COGNITO_SECRET' (str)
Stored 'SETTLEMENT_DISCOVERY_URL' (str)


In [12]:
from helpers.utils import put_ssm_parameter, SSM_SETTLEMENT_AGENT_ARN

put_ssm_parameter(SSM_SETTLEMENT_AGENT_ARN, SETTLEMENT_AGENT_ARN)



In [13]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [14]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [15]:
fetch_agent_card(settlement_agent_arn)

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'Intelligent agent for automated auto insurance claims processing and settlement decisions',
 'name': 'AutoInsuranceClaimsSettlementAgent',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [{'description': 'Verifies applicable coverage limits against policy terms.\n\nArgs:\n    claim_number: Unique claim identifier\n    estimated_cost: Total estimated repair cost\n    \nReturns:\n    Coverage verification details with policy limits and applicability',
   'id': 'verify_coverage_limits',
   'name': 'verify_coverage_limits',
   'tags': []},
  {'description': 'Applies policy deductible to calculate net insurance payout.\n\nArgs:\n    estimated_cost: Total repair estimate\n    deductible: Policy deductible amount\n    \nReturns:\n    Detailed breakdown of adjusted amounts',
   'id': 'apply_deductible_adjustments',
   'name': 'apply_deductible_adjust

#### Test agents

Now, let's invoke the first agent, using A2A:


In [16]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

In [17]:
appraisal_json = {
  "estimate_metadata": {
    "claim_number": "CL-2023-1156789",
    "estimate_date": "2025-10-20T20:31:51.627755",
    "disclaimer": "This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates."
  },
  "vehicle_information": {
    "year": 2020,
    "make": "Honda",
    "model": "Accord",
    "vin": "1HGCV2F35LA007149",
    "mileage": 35650,
    "vehicle_type": "Car"
  },
  "damage_assessment": {
    "damaged_components": [
      {
        "component": "Rear bumper",
        "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, brake lights damaged",
        "estimated_cost": 1150.0
      },
      {
        "component": "Trunk",
        "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, brake lights damaged",
        "estimated_cost": 1850.0
      },
      {
        "component": "Taillights",
        "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, brake lights damaged",
        "estimated_cost": 450.0
      }
    ],
    "subtotal": 3450.0
  },
  "cost_adjustments": {
    "multipliers": [],
    "adjustment_total": 0.0
  },
  "total_estimate": {
    "amount": 3450.0,
    "currency": "USD"
  }
}
appraisal_input = json.dumps(appraisal_json)
user_prompt = f"""
Please Settle the claim for the following claim appraisal: {appraisal_input}:


"""

print("=" * 80)
print("USER PROMPT CREATED")
print("=" * 80)
print(user_prompt)
print("=" * 80)
print("\n📝 Prompt ready for agent execution")
print("\n" + "=" * 80)

USER PROMPT CREATED

Please Settle the claim for the following claim appraisal: {"estimate_metadata": {"claim_number": "CL-2023-1156789", "estimate_date": "2025-10-20T20:31:51.627755", "disclaimer": "This is a preliminary estimate. Actual repair costs may vary based on hidden damage, parts availability, and labor rates."}, "vehicle_information": {"year": 2020, "make": "Honda", "model": "Accord", "vin": "1HGCV2F35LA007149", "mileage": 35650, "vehicle_type": "Car"}, "damage_assessment": {"damaged_components": [{"component": "Rear bumper", "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, brake lights damaged", "estimated_cost": 1150.0}, {"component": "Trunk", "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, brake lights damaged", "estimated_cost": 1850.0}, {"component": "Taillights", "damage_description": "Rear bumper dented and paint scratched, trunk lid misaligned, no frame damage, 

In [18]:
settlement_agent_arn

'arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/settlement_assistant-3t3GvOE7iu'

In [19]:
result = await send_sync_message(settlement_agent_arn, user_prompt)
formatted_output = format_agent_response(result)
print(formatted_output)

Generated session ID: c9ee5689-992b-47af-81f6-f59e9793f1e2
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='Intelligent agent for automated auto insurance claims processing and settlement decisions' documentation_url=None icon_url=None name='AutoInsuranceClaimsSettlementAgent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Verifies applicable coverage limits against policy terms.\n\nArgs:\n    claim_number: Unique claim identifier\n    estimated_cost: Total estimated repair cost\n    \nReturns:\n    Coverage verification details with policy limits and applicability', examples=None, id='verify_coverage_limits', input_modes=None, name='verify_coverage_limits', output_modes=None, security=None, tags=[]), AgentSkill(de

In [20]:
🎯 KEY FEATURES:
1. Complete Tool Suite (@tool decorated)
✅ verify_coverage_limits - Policy validation
✅ apply_deductible_adjustments - Financial calculations
✅ evaluate_claim_decision - Decision engine (APPROVE/DENY/INVESTIGATE)
✅ create_claim_documentation - Audit trail generation
✅ generate_settlement_output - Formatted output production
2. Intelligent Decision Logic
INVESTIGATE outcome triggered for:
Possible frame damage detected ⚠️
Claims > $5,000
Includes specific investigation requirements
3-5 day timeline provided
3. Comprehensive Output
✅ Settlement summary with line-item breakdown
✅ Payment authorization with codes
✅ Required documentation checklist
✅ Conditions and next steps
✅ Contact information
4. Audit Compliance
Complete documentation trail
Policy citations for denials
Regulatory compliance markers
Timestamp and processing history
Expected Output: The claim will be flagged for INVESTIGATE due to "possible frame damage" mentioned in the damage description, requiring adjuster inspection before final settlement! 

SyntaxError: invalid character '🎯' (U+1F3AF) (2577767818.py, line 1)